In [1]:
from apscheduler.schedulers.background import BackgroundScheduler as Scheduler
import requests
import json
import pandas as pd
# from os import path
from time import time

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def lookupDict(dic, key, *keys):
    if keys:
        return lookupDict(dic.get(key, {}), *keys)
    return dic.get(key)

In [2]:
database_dir = "C:\\Users\\AAN0709\\Toyota Motor Europe\\Electrified Powertrain Research (EPR) - Research Institute Mapping\\3. Research Institute Mapping\\DATABASES\\IEEE Xplore\\API\\"
cep_api_dir = "/home/aan0709@tmme/pcu-research-mapping/data/IEEE_test/"
# cep_api_dir = database_dir

api = "http://ieeexploreapi.ieee.org/api/v1/search/articles?format=json&apikey="
key = "thbuwwkrvadxd9253ruc3egm"    # 200 calls per day, active
# key = "cpe4yazv44u57jxtuubr7tv9"    # only one call available
# key = "e3gfcxmf68z6x3k7ngcdepcu"    # Inactive API

max_records = 200
start_record = 1

meta_data = ["onboard charger OR on-board charger OR integrated charger",
             "dc-dc converter OR dc/dc converter",
             "inverter OR dc/ac converter OR dc-ac converter",    # 16797 -> 16722 -> 16325
             "gallium nitride",    # searching for GaN makes more harm than good
             "silicon carbide"]    # same, SiC corresponds to something else most of the time

parameters = {
    "content_type": "Journals",
    "start_year": "1970",
    "max_records": max_records,  # Can go to max 200
    "sort_field": "article_number",
    "sort_order": "asc",  # Oldest first!
    # "index_terms": index_terms[2],
    "meta_data": meta_data[0],
    "start_record": start_record # starts at 1
}

In [3]:
def myfn():
    start_fn = time()

    # global df
    global count
    global parameters
    global start_record
    global max_records
    global force_terminate
    global df

    # ~~~ CALL ~~~
    parameters["start_record"] = start_record
    start_req = time()
    responseIEEE = requests.get(api + key, params=parameters)
    end_req = time()
    count += 1
    if responseIEEE.status_code != 200:
        print(responseIEEE.status_code)
    # ~~~ CALL ~~~
    try:
        # print("Entered TRY")
        json_file = responseIEEE.json()["articles"]
        data = json.dumps(json_file, sort_keys=True, indent=4)
        data = json.loads(data)
        df_call = pd.json_normalize(data)
        df = pd.concat([df, df_call])
        end_fn = time()
        print("Excel No = %d" % count)
        print("Papers = %d - %d" % (start_record, start_record-1 + max_records))
        print("Downloaded in %0.2f seconds." % (end_req - start_req))
        # print("Time for Function= " + str(end_fn - start_fn) + " seconds")
        print("// ----- ----- ----- //")
        start_record += max_records
    except KeyError:
        print("Entered KeyError")
        print("Response is empty, terminating program")
        force_terminate = True
        sch.remove_job("responseIEEE")

count = 0
force_terminate = False
        
df = pd.DataFrame()

# IEEE limitations: 10 calls per second, 200 calls per day
sch = Scheduler()
# For 5 second interval and 200 calls per day, it takes approximately 16 minutes
# But to download 200 entities, it needs almost ~10 seconds
sch.add_job(myfn, 'interval', seconds=15, id="responseIEEE")
sch.start()

# This code will be executed after the scheduler has started
try:
    print("Scheduler started!")
    print("// ----- ----- ----- //")
    while not force_terminate:
        pass
except (KeyboardInterrupt, SystemExit):
    print("\n\nProgram terminated.\nError occurred.\n")
    sch.shutdown()

print("\n\nProgram terminated correctly.")
sch.shutdown()

Scheduler started!
// ----- ----- ----- //
Excel No = 1
Papers = 1 - 200
Downloaded in 9.31 seconds.
// ----- ----- ----- //
Excel No = 2
Papers = 201 - 400
Downloaded in 7.89 seconds.
// ----- ----- ----- //
Excel No = 3
Papers = 401 - 600
Downloaded in 1.06 seconds.
// ----- ----- ----- //


Program terminated.
Error occurred.



Program terminated correctly.


SchedulerNotRunningError: Scheduler is not running

In [4]:
df.reset_index(inplace=True, drop=True)
df.to_excel(cep_api_dir + "OBC" + ".xlsx")

In [10]:
cep_api_dir = "/home/aan0709@tmme/pcu-research-mapping/data/IEEE_test/"

In [5]:
df

,abstract,abstract_url,access_type,article_number,citing_paper_count,citing_patent_count,content_type,doi,end_page,html_url,...,publication_title,publication_year,publisher,rank,start_page,title,volume,authors.authors,index_terms.author_terms.terms,index_terms.ieee_terms.terms
0,This paper presents a 950-MHz wireless power t...,https://ieeexplore.ieee.org/document/1564343/,LOCKED,1564343,310,2,Journals,10.1109/JSSC.2005.858620,41,https://ieeexplore.ieee.org/document/1564343/,...,IEEE Journal of Solid-State Circuits,2006,IEEE,1,35,A 950-MHz rectifier circuit for sensor network...,41,"[{'affiliation': 'Mobile Commun. Lab., Corp. R...","[Bias voltage distributor, effective threshold...","[Rectifiers, Circuits, Wireless sensor network..."
1,This paper proposes a high-efficiency bidirect...,https://ieeexplore.ieee.org/document/1703726/,LOCKED,1703726,88,0,Journals,10.1109/TIA.2006.880847,1332,https://ieeexplore.ieee.org/document/1703726/,...,IEEE Transactions on Industry Applications,2006,IEEE,2,1321,Analysis and Design of a New High-Efficiency B...,42,"[{'affiliation': 'Univ. de Caxias do Sul', 'au...","[Soft switching, uninterruptible power supply,...","[Pulse width modulation converters, Uninterrup..."
2,This paper presents a new compact CMOS Li-Ion ...,https://ieeexplore.ieee.org/document/4155033/,LOCKED,4155033,75,0,Journals,10.1109/TCSI.2007.890605,712,https://ieeexplore.ieee.org/document/4155033/,...,IEEE Transactions on Circuits and Systems I: R...,2007,IEEE,3,705,New Compact CMOS Li-Ion Battery Charger Using ...,54,"[{'affiliation': 'Dept. of Electron. Eng., Nat...","[Charge pump, charger circuit, Li-Ion battery]","[Batteries, Charge pumps, Voltage, Circuits, E..."
3,This paper indicates the market for a van-type...,https://ieeexplore.ieee.org/document/4157643/,LOCKED,4157643,1,0,Journals,10.1109/TIA.1974.349203,552,https://ieeexplore.ieee.org/document/4157643/,...,IEEE Transactions on Industry Applications,1974,IEEE,4,544,An AC Drive Electric Vehicle,IA-10,"[{'affiliation': 'Linear Alpha, Inc., Skokie, ...",NaN,"[Electric vehicles, Batteries, Pulse modulatio..."
4,Wireless communication now has the speed and p...,https://ieeexplore.ieee.org/document/4277880/,LOCKED,4277880,72,6,Journals,10.1109/JSSC.2007.900284,1731,https://ieeexplore.ieee.org/document/4277880/,...,IEEE Journal of Solid-State Circuits,2007,IEEE,5,1723,A Highly Integrated Power Management IC for Ad...,42,"[{'affiliation': 'Qualcomm Inc., San Diego', '...","[Buck regulator, DC-DC converter, LDO, linear ...","[Energy management, Application specific integ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,"In this article, a novel electric-drive-recons...",https://ieeexplore.ieee.org/document/9669134/,LOCKED,9669134,0,0,Journals,10.1109/TPEL.2022.3140410,6555,https://ieeexplore.ieee.org/document/9669134/,...,IEEE Transactions on Power Electronics,2022,IEEE,405,6544,Electric-Drive-Reconstructed Onboard Charger f...,37,[{'affiliation': 'School of Electrical Enginee...,[Electric-drive-reconstructed onboard charger ...,"[Topology, Batteries, Windings, Switches, Rene..."
405,In order to widen voltage gain of bidirectiona...,https://ieeexplore.ieee.org/document/9672731/,LOCKED,9672731,0,0,Journals,10.1109/TPEL.2022.3140902,6985,https://ieeexplore.ieee.org/document/9672731/,...,IEEE Transactions on Power Electronics,2022,IEEE,406,6972,A Novel Bidirectional Five-Level Multimode <it...,37,[{'affiliation': 'Department of Electrical Eng...,"[Bidirectional on-board charger (OBC), CLLC re...","[Bridge circuits, Voltage control, Topology, M..."
406,To accurately simulate electric vehicle DC fas...,https://ieeexplore.ieee.org/document/9675057/,OPEN_ACCESS,9675057,0,0,Journals,10.23919/CJEE.2021.000034,24,https://ieeexplore.ieee.org/document/9675057/,...,Chinese Journal of Electrical Engineering,2021,CMP,407,15,Python supervised co-simulation for a day-long...,7,[{'affiliation': 'Department of Electrical Sus...,"[Power quality, h

In [15]:
df.reset_index(inplace=True, drop=True)

In [4]:
df["article_number"]

0      1019967
1      1021297
2       103420
3       103437
4      1035139
        ...   
176    9693909
177     974372
178      97767
179     982263
180     993162
Name: article_number, Length: 181, dtype: object

In [14]:
df

,abstract,abstract_url,access_type,article_number,citing_paper_count,citing_patent_count,content_type,doi,end_page,html_url,...,publication_title,publication_year,publisher,rank,start_page,title,volume,authors.authors,index_terms.ieee_terms.terms,index_terms.author_terms.terms
0,"Presents analytical results, numerical simulat...",https://ieeexplore.ieee.org/document/1001951/,LOCKED,1001951,14,0,Journals,10.1109/TCSI.2002.1001951,631,https://ieeexplore.ieee.org/document/1001951/,...,IEEE Transactions on Circuits and Systems I: F...,2002,IEEE,1,621,Modeling of PWM inverter-supplied AC drives at...,49,[{'affiliation': 'Dept. of Electr. & Comput. E...,"[Pulse width modulation inverters, Pulse width...",NaN
1,A neural-network-based implementation of space...,https://ieeexplore.ieee.org/document/1003415/,LOCKED,1003415,97,10,Journals,10.1109/TIA.2002.1003415,669,https://ieeexplore.ieee.org/document/1003415/,...,IEEE Transactions on Industry Applications,2002,IEEE,2,660,A neural-network-based space-vector PWM contro...,38,"[{'affiliation': 'Dept. of Electr. Eng., Tenne...","[Space vector pulse width modulation, Voltage,...",NaN
2,This paper presents an integrated scheme for p...,https://ieeexplore.ieee.org/document/1003417/,LOCKED,1003417,28,0,Journals,10.1109/TIA.2002.1003417,687,https://ieeexplore.ieee.org/document/1003417/,...,IEEE Transactions on Industry Applications,2002,IEEE,3,679,An integrated approach for the protection of s...,38,[{'affiliation': 'Dept. of Electr. & Comput. S...,"[Inverters, Power system protection, Circuit f...",NaN
3,A low-cost inverter capable of operating bifil...,https://ieeexplore.ieee.org/document/1003418/,LOCKED,1003418,9,0,Journals,10.1109/TIA.2002.1003418,694,https://ieeexplore.ieee.org/document/1003418/,...,IEEE Transactions on Industry Applications,2002,IEEE,4,688,A novel low-cost RCD snubber for bifilar-wound...,38,"[{'affiliation': 'Leicester Univ., UK', 'autho...","[Snubbers, Inverters, Brushless DC motors, Ind...",NaN
4,This paper presents an integrated approach for...,https://ieeexplore.ieee.org/document/1003419/,LOCKED,1003419,14,0,Journals,10.1109/TIA.2002.1003419,705,https://ieeexplore.ieee.org/document/1003419/,...,IEEE Transactions on Industry Applications,2002,IEEE,5,695,A variable band universal flux/charge modulato...,38,[{'affiliation': 'Dept. of Electr. & Comput. S...,"[Pulse width modulation inverters, Hysteresis,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,This paper focuses on current control in a per...,https://ieeexplore.ieee.org/document/8649814/,LOCKED,8649814,17,0,Journals,10.1109/TCYB.2019.2897653,3230,https://ieeexplore.ieee.org/document/8649814/,...,IEEE Transactions on Cybernetics,2020,IEEE,11196,3218,Neural-Network Vector Controller for Permanent...,50,[{'affiliation': 'Department of Electrical and...,"[Artificial neural networks, Machine vector co...","[Approximate dynamic programming (ADP), neural..."
196,A nondestructive column-selection-enabled 10T ...,https://ieeexplore.ieee.org/document/8649816/,LOCKED,8649816,4,0,Journals,10.1109/TVLSI.2019.2898346,1474,https://ieeexplore.ieee.org/document/8649816/,...,IEEE Transactions on Very Large Scale Integrat...,2019,IEEE,11197,1470,Column-Selection-Enabled 10T SRAM Utilizing Sh...,27,[{'affiliation': 'Department of Electronic Tec...,"[Computer architecture, Microprocessors, SRAM ...","[10T SRAM, bit-interleaving, column selection,..."
197,A method for fast calculating the high-frequen...,https://ieeexplore.ieee.org/document/8651397/,LOCKED,8651397,13,0,Journals,10.1109/TEC.2019.2901299,1592,https://ieeexplore.ieee.org/document/8651397/,...,IEEE Transactions on Energy Conversion,2019,IEEE,11198,1581,Fast Calculation of Carrier Harmonic Loss in P...,34,[{'affiliation': 'Department of Energy and Ele...,"[Pulse width modulation, Rotors, Employee welf...","[Permanent magnet eddy current loss, interior ..."
198,This paper presents a fault diagnosis method a...,https://iee